In [1]:
# import x01_generate_notes
import x02_generate_midi
import x03_generate_wav
import x04_generate_wav_processed
import x05_generate_spectrogram
import x06_generate_label

In [2]:
!mkdir notes
!mkdir midi
!mkdir wav
!mkdir wav_processed
!mkdir spectrogram
!mkdir label
!rm ./notes/*
!rm ./midi/*
!rm ./wav/*
!rm ./wav_processed/*
!rm ./spectrogram/*
!rm ./label/*

mkdir: notes: File exists
mkdir: midi: File exists
mkdir: wav: File exists
mkdir: wav_processed: File exists
mkdir: spectrogram: File exists
mkdir: label: File exists
rm: ./wav_processed/*: No such file or directory
rm: ./spectrogram/*: No such file or directory
rm: ./label/*: No such file or directory


In [3]:
import numpy as np
import random

# this generates what time play what note and saves it to ./notes/

def generate_note_file():
    # could be summarised
    sound_id = ''.join(random.choice('0123456789ABCDEF') for i in range(16))
    
    # t = 2 means the 1st second, since the tempo is 120bpm.
    # this model does not assume any periodicity in transcription
    t = 0.1
    note_time_array = []
    
    chord_holder = []
    
    while t < 12:
        t += 0.25 + np.random.uniform(0,1.1)**5
        
        notes_in_this_chord = []
        
        if len(chord_holder) and np.random.uniform() > 0.75:
            notes_in_this_chord = chord_holder
            if np.random.uniform() < 1/len(chord_holder)**4:
                new_note = np.random.randint(24)
                if new_note not in notes_in_this_chord:
                    notes_in_this_chord.append(new_note)
        
        elif np.random.uniform() > 0.9:
            # generate one note
            new_note = np.random.randint(24)
            
        elif np.random.uniform() > 0.4 or t > 18:
            # random generation
            
            if np.random.uniform() > 0.2:
                new_note = np.random.randint(24)
                if new_note not in notes_in_this_chord:
                    notes_in_this_chord.append(new_note)

            if np.random.uniform() > 0.2:
                new_note = np.random.randint(24)
                if new_note not in notes_in_this_chord:
                    notes_in_this_chord.append(new_note)

            if np.random.uniform() > 0.3:
                new_note = np.random.randint(24)
                if new_note not in notes_in_this_chord:
                    notes_in_this_chord.append(new_note)

            if np.random.uniform() > 0.3:
                new_note = np.random.randint(24)
                if new_note not in notes_in_this_chord:
                    notes_in_this_chord.append(new_note)
                
        else:
            # favouring usual chords
            new_note = np.random.randint(24)
            notes_in_this_chord.append(new_note)
            
            if np.random.uniform() > 0.8:
                new_note = np.random.randint(24-7)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+3)
                notes_in_this_chord.append(new_note+7)
                
            elif np.random.uniform() > 0.8:
                new_note = np.random.randint(24-7)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+4)
                notes_in_this_chord.append(new_note+7)
                
            elif np.random.uniform() > 0.9:
                new_note = np.random.randint(24-7)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+3)
                notes_in_this_chord.append(new_note+6)
                
            elif np.random.uniform() > 0.9:
                new_note = np.random.randint(24-8)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+4)
                notes_in_this_chord.append(new_note+8)
                
            elif np.random.uniform() > 0.95:
                new_note = np.random.randint(24-12)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+3)
                notes_in_this_chord.append(new_note+7)
                notes_in_this_chord.append(new_note+12)
                
            elif np.random.uniform() > 0.95:
                new_note = np.random.randint(24-12)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+4)
                notes_in_this_chord.append(new_note+7)
                notes_in_this_chord.append(new_note+12)
                
            elif np.random.uniform() > 0.95:
                new_note = np.random.randint(24-12)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+3)
                notes_in_this_chord.append(new_note+6)
                notes_in_this_chord.append(new_note+12)
                
            elif np.random.uniform() > 0.95:
                new_note = np.random.randint(24-12)
                notes_in_this_chord.append(new_note)
                notes_in_this_chord.append(new_note+4)
                notes_in_this_chord.append(new_note+8)
                notes_in_this_chord.append(new_note+12)
                
            else:
                # generate one note
                new_note = np.random.randint(24)
                
        chord_holder = notes_in_this_chord
        for note in notes_in_this_chord:
            note_time_array.append([t, note])
            #print(t)
            #print(notes_in_this_chord)
                
#     #print(note_time_array)
    #print(np.shape(note_time_array))
    np.save("./notes/" + sound_id, np.array(note_time_array))

In [4]:
for _ in range(5):
    generate_note_file()

In [5]:
import glob

# for every (time-note) file in ./notes, generate .mid file 

notes = glob.glob("./notes/*.npy")
print(len(notes))

for filename in notes:
    sound_id = filename[-20:-4]
    time_note_array = np.load(filename)
    time_array, note_array = np.transpose(time_note_array)
    x02_generate_midi.generate_midi_file(sound_id, time_array, np.array(note_array, dtype=int))

5


In [6]:
import glob

# for every .mid file generate the .wav file with fluidsynth

notes = glob.glob("./midi/*")
print(len(notes))
for filename in notes:
    x03_generate_wav.convert_midi_to_signal(filename)

5


In [7]:
import glob
# for every .wav file do some processing and generate another .wav file

files = glob.glob('./wav/*')
save_location = './wav_processed/'
print(len(files))
for filename in files:
    x04_generate_wav_processed.process_wav(filename, save_location)

5


In [8]:
import soundfile as sf
import sounddevice as sd

audio, sr = sf.read(files[0])

sd.play(audio, samplerate=sr)

In [9]:
import glob
files = glob.glob('./wav_processed/*')
print(len(files))
for filename in files:
    x05_generate_spectrogram.generate_spectrogram(filename)

5


In [10]:
# import glob
# files = glob.glob('../*.wav')
# #print(len(files))
# for filename in files:
#     x05_generate_spectrogram.generate_spectrogram(filename)

In [11]:
import soundfile as sf
import sounddevice as sd

audio, sr = sf.read(files[0])

sd.play(audio, samplerate=sr)

In [12]:
# for every time-note file .npy generate .npy labels

import glob

notes = glob.glob("./notes/*.npy")
print(len(notes))

for filename in notes:
#     sound_id = filename[-20:-4]
#     time_note_array = np.load(filename)
#     time_array, note_array = np.transpose(time_note_array)
    x06_generate_label.generate_label(filename)

5


In [13]:
# !ls ./notes/
# !ls ./midi/
# !ls ./wav/
# !ls ./wav_processed/
# !ls ./spectrogram/
# !ls ./label/